#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [40]:
import pandas as pd
import numpy as np

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

In [41]:
merged_data = pd.merge(brands, finance, on='product_id', how="outer")
merged_data = pd.merge(merged_data, info, on='product_id', how="outer")
merged_data = pd.merge(merged_data, reviews, on='product_id', how="outer")
merged_data.dropna(inplace=True)

print(merged_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 1 to 3178
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     3117 non-null   object 
 1   brand          3117 non-null   object 
 2   listing_price  3117 non-null   float64
 3   sale_price     3117 non-null   float64
 4   discount       3117 non-null   float64
 5   revenue        3117 non-null   float64
 6   product_name   3117 non-null   object 
 7   description    3117 non-null   object 
 8   rating         3117 non-null   float64
 9   reviews        3117 non-null   float64
dtypes: float64(6), object(4)
memory usage: 267.9+ KB
None


In [42]:
custom_labels = ['Budget', 'Average', 'Expensive', 'Elite']

merged_data['price_label'] = pd.qcut(merged_data['listing_price'], q=4, labels=custom_labels)

adidas_vs_nike = merged_data.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"),
        mean_revenue=("revenue", "mean")
).round(2).reset_index(drop=True)

print(adidas_vs_nike)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45


In [43]:
max(merged_data["description"].str.len())

merged_data['description_length'] = merged_data['description'].str.len()

lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

labels = ["100", "200", "300", "400", "500", "600", "700"]

merged_data['description_length'] = pd.cut(merged_data['description_length'], bins=lengthes, labels=labels)

description_lengths = merged_data.groupby('description_length', as_index=False).agg(
    mean_rating = ('rating', 'mean'),
    num_reviews = ('reviews', 'count')).round(2)

print(description_lengths)

  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15


In [44]:
mylist = "shoe*|trainer*|foot*"

# Filter for footwear products
shoes = merged_data[merged_data["description"].str.contains(mylist)]

# Filter for clothing products
clothing = merged_data[~merged_data.isin(shoes["product_id"])]
clothing.dropna(inplace=True)

product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

print(product_types)

   num_clothing_products  ...  median_footwear_revenue
0                    478  ...                   3073.3

[1 rows x 4 columns]
